In [1]:
import os

In [2]:
%pwd

'd:\\Bappy\\YouTube\\End-to-end-ML-Project-with-DVC-MLflow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Bappy\\YouTube\\End-to-end-ML-Project-with-DVC-MLflow'

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class RandomForestModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    max_depth: int
    random_state: int
    n_estimators: int
    min_samples_split: int
    target_column: str

In [8]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_trainer_config(self) -> RandomForestModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForest
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = RandomForestModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            max_depth = params.max_depth,
            random_state = params.random_state,
            n_estimators = params.n_estimators,
            min_samples_split = params.min_samples_split,
            target_column = schema.name
            
        )

        return model_trainer_config

In [10]:
import pandas as pd
import os
from mlProject import logger
from sklearn.ensemble import RandomForestRegressor
import joblib

In [11]:
class ModelTrainer:
    def __init__(self, config: RandomForestModelTrainerConfig):
        self.config = config


    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]


        rf = RandomForestRegressor(max_depth=self.config.max_depth,random_state=self.config.random_state, n_estimators=self.config.n_estimators,min_samples_split= self.config.min_samples_split)
        rf.fit(train_x, train_y)

        joblib.dump(rf, os.path.join(self.config.root_dir, "randomForest.joblib"))


In [12]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2023-06-19 12:21:55,800: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-06-19 12:21:55,807: INFO: common: yaml file: params.yaml loaded successfully]
[2023-06-19 12:21:55,810: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-06-19 12:21:55,811: INFO: common: created directory at: artifacts]
[2023-06-19 12:21:55,812: INFO: common: created directory at: artifacts/model_trainer]


C:\Users\bokti\AppData\Local\Temp\ipykernel_14848\1846430960.py:19: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(train_x, train_y)
